In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AdamW
import torch
import pandas as pd

In [10]:
#設定一個list
set1_df = pd.read_csv("First_Phase_ReleaseCorrectionTut\First_Phase_Release(Correction)\Tutorial\opendid_set1.tsv",sep='\t',header=None,names=['file_name','start','content','PHI'])
ans_df = pd.read_csv("First_Phase_ReleaseCorrectionTut\First_Phase_Release(Correction)\Tutorial\opendid_valid.tsv",sep='\t',header=None,names=['file_name','start','content'])
set1_content_list = set1_df['content'].tolist()
set1_PHI_list = set1_df['PHI'].tolist()
ans_content_list = ans_df['content'].tolist()
# 假設 set1_content_list 包含的是其他類型的元素，比如整數或其他類型的變數
set1_content_list = [str(item) for item in set1_content_list]
# 假設 set1_PHI_list 包含的是其他類型的元素，比如整數或其他類型的變數
set1_PHI_list = [str(item) for item in set1_PHI_list]
print(set1_PHI_list)
# 訓練資料集

['IDNUM: 09F016547J', 'MEDICALRECORD: 091016.NMT', 'PATIENT: SIZAR, HOWARD', 'IDNUM: 09F01654', 'STREET: Runford', 'CITY: RENMARK\\nSTATE: TAS\\nZIP: 5084', 'PHI: NULL', 'DATE: 24/8/1993=>1993-08-24', 'PHI: NULL', 'TIME: 28/08/2013 at 08:26=>2013-08-28T08:26', 'DEPARTMENT: St Vincent\\nHOSPITAL: BATLOW/ADELONG MULTI PURPOSE SERVICE', 'DOCTOR: JAXON AL-KARSTEN', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'DOCTOR: RF\\nDOCTOR: BL\\nDATE: 29.10.63=>2063-10-29', 'DOCTOR: A Hobday', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', 'PHI: NULL', '

In [11]:
# 初始化模型和tokenizer
model_name = "EleutherAI/gpt-j-6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 設置訓練相關參數
MAX_LENGTH = 1024
VOCAB_SIZE = 50256
BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 3e-5

# 訓練模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optim = AdamW(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    for i in range(0, len(set1_content_list), BATCH_SIZE):
        batch = set1_content_list[i:i + BATCH_SIZE]
        inputs = tokenizer(batch, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
        inputs.to(device)
        outputs = tokenizer(set1_PHI_list[i:i + BATCH_SIZE], padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
        outputs.to(device)

        optim.zero_grad()
        model.train()
        loss = model(**inputs, labels=outputs).loss
        loss.backward()
        optim.step()

# 儲存模型
model.save_pretrained("gptj_trained_model")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 268435456 bytes.

In [ ]:
# 載入訓練後的模型
model_path = "gptj_trained_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

In [ ]:
# 使用模型回答新的問題
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for c in ans_content_list:
    inputs = tokenizer(c, return_tensors="pt", truncation=True, max_length=1024)
    inputs.to(device)
    outputs = model.generate(**inputs, max_length=1024)
    print(f"{c}")
    print(f"{tokenizer.decode(outputs[0], skip_special_tokens=True)}")
